In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import cv2
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# 1. DepthwiseConv2D 에러 방지 패치
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.utils import CustomObjectScope

def fixed_depthwise_conv2d(**kwargs):
    if 'groups' in kwargs: del kwargs['groups']
    return DepthwiseConv2D(**kwargs)

# 2. 모델 로드
with CustomObjectScope({'DepthwiseConv2D': fixed_depthwise_conv2d}):
    from tensorflow.keras.models import load_model
    model = load_model('/content/keras_model.h5', compile=False)

class_names = [line.strip() for line in open('/content/labels.txt', 'r').readlines()]

# 3. 웹캠 함수
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      const button = document.createElement('button');
      button.textContent = '분류하기 (클릭)';
      div.appendChild(button);
      await new Promise((resolve) => button.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth; canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f: f.write(binary)
  return filename

# 4. 실행부
try:
  while True:
    filename = take_photo()
    img = cv2.imread(filename)
    img = cv2.resize(img, (224, 224))
    img = np.asarray(img, dtype=np.float32).reshape(1, 224, 224, 3)
    img = (img / 127.5) - 1

    # 최신 방식의 예측 (verbose=0으로 경고 방지)
    prediction = model.predict(img, verbose=0)
    index = np.argmax(prediction)
    print(f"결과: {class_names[index]}, 확률: {prediction[0][index]*100:.2f}%")
except Exception as err:
  print(f"오류 발생: {err}")

<IPython.core.display.Javascript object>

결과: 0 강아지, 확률: 98.45%


<IPython.core.display.Javascript object>

결과: 3 배경, 확률: 99.39%


<IPython.core.display.Javascript object>

...

결과: 0 강아지, 확률: 89.38%


<IPython.core.display.Javascript object>

결과: 0 강아지, 확률: 97.76%


<IPython.core.display.Javascript object>

KeyboardInterrupt: 